## Calculate metrics: recall and precision, for given detector

In [9]:
import os
import glob
from pathlib import Path
import argparse

from collections import Counter

import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd
import skimage.io

In [10]:
def load_class_names(names_file):
    '''
    Return id_to_class and class_to_id mappings
    '''
    id_to_class = {}
    class_to_id = {}
    with open(names_file, 'r+') as file:
        for i, line in enumerate(file):
            id_to_class[i] = line.strip()
            class_to_id[line.strip()] = i
    return id_to_class, class_to_id

In [31]:
def load_info_xml(filename, fmt='voc'):
    '''
    Load bounding boxes with class_id, coords and confidence from .xml file
    :args:
        filename -- name of file with info about bboxes
        fmt -- format of bboxes: 'voc' -- read PASCAL VOC-like .xml file with bounding boxes
    :return:
        info -- dict with keys: 'class_id', 'coords', 'conf' (None if ground truth) 
    '''
    e = ET.parse(filename).getroot()
    bboxes = []
    for obj in e.findall('object'):
        class_id = CLASS_TO_ID[obj.find('name').text]
        bbox = obj.find('bndbox')
        x1 = float(bbox.find('xmin').text)
        y1 = float(bbox.find('ymin').text)
        conf = bbox.find('conf')
        if conf is not None:
            conf = float(conf.text)
        if fmt == 'voc':
            x2 = float(bbox.find('xmax').text)
            y2 = float(bbox.find('ymax').text)
            bboxes.append({'class_id': class_id, 
                           'coords': (x1, y1, x2, y2), 
                           'conf': conf})
        elif fmt == 'coco':
            w = float(bbox.find('w').text)
            h = float(bbox.find('h').text)
            bboxes.append({'class_id': class_id, 
                           'coords': (x1, y1, w, h), 
                           'conf': conf})
    return bboxes

In [32]:
def load_info_txt(filename, img_size):
    '''
    Load bounding boxes with class_id, coords and confidence from .txt file in YOLO-format
    :args:
        filename -- name of file with info about bboxes
        img_size -- original image (height, width)
    :return:
        info -- dict with keys: 'class_id', 'coords', 'conf' (None if ground truth) 
    '''
    bboxes = []
    img_height = img_size[0]
    img_width = img_size[1]
    with open(filename, 'r+') as file:
        for line in file:
            class_id, center_x, center_y, width, height = [float(x) for x in line.split(' ')]
            # to original size
            center_x *= img_width
            center_y *= img_height
            width *= img_width
            height *= img_height
            # to (x1, y1, x2, y2) corners format
            w2 = width / 2
            h2 = height / 2
            x1 = center_x - w2
            y1 = center_y - h2
            x2 = center_x + w2
            y2 = center_y + h2
            # confidence can be added later
            bboxes.append({'class_id': int(class_id), 
                           'coords': (x1, y1, x2, y2), 
                           'conf': None})
    return bboxes

In [33]:
def iou(boxA, boxB):
    '''
    Calculate the Intersection over Union of two bounding boxes.
    Boxes must be represented as (x1, y1, x2, y2): upper left and lower right corners
    '''
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

In [41]:
def metrics_per_class(gt_dir, pred_dir, fmt, names_file, yolo_format=False,
                      iou_thresh=0.5, conf_thresh=0.5, save_to_file=True):
    '''
    Calculate metrics (precision and recall) values for each class
    :args:
        classes -- dict with IDs of all classes as keys 
                   and names of classes as values (e.g. {0: 'butter', 1:'bread', ..})
        gt_dir -- folder with ground truth bounding boxes files
        pred_dir -- folder with predicted bounding boxes files
        iou_thresh -- threshold of IoU for predicted and GT bboxex overlap
        conf_thresh -- confidence threshold strting from which bbox is taken into account
        save_to_file -- if True, save the result table to a file
    :return:
        metrics_df -- pd.DataFrame with class_names in columns
        
    NOTE: number of files with bboxes must be the same in the gt_dir and in the pred_dir, 
    each ground truth file must match the name of a predicted file
    '''
    
    ID_TO_CLASS, CLASS_TO_ID = load_class_names(names_file)

    class_all = Counter()  # per class counts along all test sample
    class_tp = Counter()  # per class True Positives
    class_fp = Counter()  # per class False Positives
    strange_images = []
    
    # for every single image calculate precision and recall
    for gt_file, pred_file in zip(glob.glob(gt_dir+os.sep+f'*.{fmt}'), 
                                  glob.glob(pred_dir+os.sep+f'*.{fmt}')):
        if fmt == 'txt':
            img_size = skimage.io.imread(gt_file[:-4] + '.jpg').shape
            gt_info = load_info_txt(gt_file, img_size)
            # MUST BE CHANGED IN CASE OF ANOTHER FORMAT
            pred_info = load_info_txt(pred_file, img_size)
        elif fmt == 'xml':
            gt_info, pred_info = load_info_xml(gt_file), load_info_xml(pred_file)
        else:
            raise ValueError('Format ' + fmt + ' is not supported')
        print(gt_file, pred_file)
        pred_info = list(filter(lambda bbox: (bbox['conf'] is None) 
                                 or (bbox['conf'] >= conf_thresh), pred_info))

        iou_matrix = np.array([np.array([iou(bbox1['coords'], bbox2['coords']) 
                                          for bbox2 in pred_info]) for bbox1 in gt_info])
        print(iou_matrix)
        iou_matrix = iou_matrix >= iou_thresh
        # can be done with broadcasting, this variant is for readability
        class_match_matrix = np.array([np.array([gt_info[i]['class_id'] == pred_info[j]['class_id'] 
                                                  for j in range(len(pred_info))]) 
                                                   for i in range(len(gt_info))])
        # IoU > than iou_thresh and classes are the same
        detect_matrix = iou_matrix & class_match_matrix
        
        gt_detections = detect_matrix.sum(axis=1)
        pred_detections = detect_matrix.sum(axis=0)
        for i, det in enumerate(gt_detections):
            class_all[gt_info[i]['class_id']] += 1
            if det > 0:
                class_tp[gt_info[i]['class_id']] += 1
        for j, det in enumerate(pred_detections):
            if det == 0:
                class_fp[pred_info[j]['class_id']] += 1
            elif det > 1:
                strange_images.append(pred_file)
        print('-------------------------------------------')

    recall = {class_id : class_tp[class_id] / class_all[class_id] 
              for class_id in class_all.keys()}
    precision = {class_id : class_tp[class_id] / (class_tp[class_id] + class_fp[class_id]) 
                 for class_id in class_all.keys()}
    metrics_df = pd.DataFrame(data=[recall, precision])
    metrics_df.columns = [ID_TO_CLASS[col] for col in metrics_df.columns]
    metrics_df.index = ['recall', 'precision']
    if save_to_file:
        metrics_df.to_excel('./metrics.xls')
    return metrics_df

In [42]:
GT_DIR = './truth/'
PRED_DIR = './preds/'
NAMES_FILE = './cards.names.txt'

In [43]:
metrics_df = metrics_per_class(GT_DIR, PRED_DIR, fmt='txt', names_file=NAMES_FILE)

./truth\cam_image2.txt ./preds\cam_image2.txt
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
-------------------------------------------
./truth\cam_image4.txt ./preds\cam_image4.txt
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
-------------------------------------------
./truth\cam_image45.txt ./preds\cam_image45.txt
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
-------------------------------------------
./truth\cam_image5.txt ./preds\cam_image5.txt
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
-------------------------------------------
./truth\cam_image6.txt ./preds\cam_image6.txt
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
-------------------------------------------
./truth\cam_image7.txt ./preds\cam_image7.txt
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
-------------------------------------------
./truth\cam_image8.txt ./preds\cam_image8.txt
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 

In [ ]:
metrics_df

In [39]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('-g', '--gt_dir', 
                        help='folder with ground truth .txt files with bounding boxes', required=True)
    parser.add_argument('-p', '--pred_dir', 
                        help='folder with predicted .txt files with bounding boxes', required=True)
    parser.add_argument('-n', '--names_file', 
                        help='.names file with names of classes', required=True)
    parser.add_argument('-y', '--yolo_format', 
                        help='if True, assume that bounding boxes are in the original YOLO format \
                        (float numbers -- relative coordinates)', default=0, required=False)
    parser.add_argument('arg', nargs='*') # use '+' for 1 or more args (instead of 0 or more)
    args = parser.parse_args()
    
    metrics_df = metrics_per_class(gt_dir=args.gt_dir, pred_dir=args.pred_dir, fmt='txt', 
                                   names_file=args.names_file, yolo_format=args.yolo_format)

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] -g GT_DIR -p PRED_DIR -n NAMES_FILE
                             [-y YOLO_FORMAT]
                             [arg [arg ...]]
ipykernel_launcher.py: error: the following arguments are required: -g/--gt_dir, -p/--pred_dir, -n/--names_file


SystemExit: 2

C:\Users\Ilya\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
